## **Basic HPO - Grid Search and Random Search**

the aim of this notebook is to make an unbiased comparison between two basic hyper parameter optimization technique:
In order to explain and undestand the main differences and strenghts.

- grid search

- random search 

Breve spiegazione delle due tecniche (con pro/contro teorici).




### **01. Setup**

#### **0. Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from src import HPO
import itertools

## TESTS

### **Dataset**

In [ ]:
# Load the California Housing Dataset and convert it into a pandas DataFrame
X, y = fetch_california_housing(return_X_y=True)

X = pd.DataFrame(
    X,
    columns=[
        "MedInc",
        "HouseAge",
        "AveRooms",
        "AveBedrms",
        "Population",
        "AveOccup",
        "Latitude",
        "Longitude",
    ],
)
y = pd.Series(y, name="target")

### **Setup**

In [ ]:
model = xgb.XGBRegressor()
data_features = X
data_target = y
hp_values = {
    "max_depth": [1, 3, 5],
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [10, 30, 50],
}
task = "regression"

In [ ]:
test_hpo = HPO(
    model=model,
    data_features=data_features,
    data_target=data_target,
    hp_values=hp_values,
    task=task,
)

### **01. GRID SEARCH**

In [ ]:
grid_search = test_hpo.hp_tuning(hpo_method="grid_search", outer_k=5, inner_k=3)

### **02. Random Search**

In [ ]:
test_hpo.hp_tuning(hpo_method="random_search", outer_k=5, inner_k=3, n_trials=9)

## DEBUG

In [ ]:
param_grid = {"max_depth": [1, 3, 5], "learning_rate": [0.01, 0.1, 0.2]}

In [ ]:
param_combinations = list(
    itertools.product(param_grid["max_depth"], param_grid["learning_rate"])
)

In [ ]:
print(param_combinations)

In [ ]:
# for combination in param_combinations:
#     model = xgb.XGBRegressor(*combination)

In [ ]:
test = HPO(model, data_features=features, data_target=target, hp_values=param_grid)

print(test.grid_search())

shape of the metrics saved in the outer loop

each element of the list:

In [ ]:
{
    "fold": fold_idx,
    "metrics": {"r2": 0.82, "mse": 0.15, "rmse": 0.387, "mae": 0.3},
    "best_config": {"max_depth": 5, "learning_rate": 0.1, "n_estimators": 200},
    "hpo_time_seconds": 120.5,
}

In [ ]:
store_metrics = [
    {"configuration": {"a": 1, "b": 2}, "results": 1},
    {"configuration": {"a": 10, "b": 20}, "results": 5},
]

best_configuration = store_metrics[0]["configuration"]
best_config_results = store_metrics[0]["results"]
for config_metrics in store_metrics:
    if config_metrics["results"] > best_config_results:
        best_configuration = config_metrics["configuration"]
        best_config_results = config_metrics["results"]
    else:
        continue


best_configuration


# TO DO

[X] i changed the output structure of the tuning parameter i need to understand it more clearly, in successive phase i can reason more on the output structure to simplify it and enache visualization

[X] create the random search method

[X] sistema output hpo_tuning 

[X]create the logger mechanism to show the steps of the classifier / model so to have a nice visualization of the model behaviour (figo come in progetto avatar il fatto di mostrare una tabella markdown nel logger con i risultati invece di printare e quindi imparare il meccanismo dei logger fatti bene) 
in modo da non dare sempre il dizionario per il lato backend ma dare il dizionario solo se viene fatto print

[ ] logger implementato correttaemente, sistema da un punto di vista visuale la struttura dei logs

[ ]create the data visualization method

[ ]create the EA method

[ ]creazione primo noteboook per mostrare funzionamento del pacchetto su grid search e random search

[ ] create the bayesian optmization method

[ ] creazione secondo noteboook per mostrare funzionamento del pacchetto su EA in dettaglio

[ ] pubblicazione pacchetto con spiegazione generale

[ ] pubblicazione primo notebook di test random e grid search 

[ ] pubblicazione secondo notebook funzionamento EA 

[ ] creazione terzo notebook per mostrare funzionamento del pacchetto su Bayesian Opt

[ ] pubblicazione terzo notebook Bayesian Opt

[ ] creare metodo per hyperparameter importance, quindi implementare logiche per selezionare, valutare l'importanza dei singoli hyperparameter

[ ] creare notebook per mostrare le strategie di HP importance

[ ] pubblicare notebook hyperparameter importance

[ ] creare automl pipeline method

[ ] creare notebook per presentare automl pipeline

FINE 



